Loading and Explorng the dataset

In [ ]:
#Importing libraries

import numpy as np
import pandas as pd

#load data
df = pd.read_csv("Orders_with_issues.csv")

#preview
print("Sample Data: \n")
print(df.head())

# summary
print("\n Column info: \n")
print(df.info())

#check missing values
print("\n Missing Values: \n")
print(df.isna().sum) # OR df.isna().sum() -> will output total number of null values in each column

#check unique shipping companies
print("\n Unique Shipping Companies: \n")
print(df['ShippingCompany'].value_counts()) # value_counts() function returns how many times each company is repeated in this column commonly used in order by scenarios

Sample Data: 

   OrderID CustomerID   OrderDate ShippedDate        ShippingCost ShipCountry  \
0   1000.0       C001  2025-05-17  2025-07-30  -57.25454602444980     Germany   
1   1001.0       C002  2025-01-26  2025-07-30              320.61      Canada   
2   1002.0       C003  2025-03-08  2025-07-30              165.17      Canada   
3   1003.0       C004  2025-03-24  2025-07-30               12.55     Germany   
4   1004.0       C005  2025-04-15  2025-07-30              186.36      Canada   

    ShipCity                 ShippingCompany  
0    Hamburg  Kiwilytics Goods Shipping LLC.  
1   Montreal                   UPS Worldwide  
2  Vancouver                 FedEx Logistics  
3     Munich            Aramex International  
4  Vancouver                 FedEx Logistics  

 Column info: 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0 

Cleaning Dates, numeric entries that don't make sense

In [18]:
#Dates are saved in csv files as numbers so we need to change them into date formates first
#Convert dates (Error types: #coerce #ignore #raise)
df['OrderDate'] = pd.to_datetime(df['OrderDate'],errors='coerce') #while errors= coerce -> if there's an error in changing value to date then put it as NULL
df['ShippedDate'] = pd.to_datetime(df['ShippedDate'], errors='coerce')

#Clean shipping cost (changing its value to numeric)
df['ShippingCost'] = pd.to_numeric(df['ShippingCost'],errors='coerce')
#grab location(index) of each shipping cost that's smaller than zero and change it to NaN(not a number) or NULL
df.loc[df['ShippingCost'] < 0, 'ShippingCost'] = np.nan
df['ShippingCost'] = df['ShippingCost'].fillna(df['ShippingCost'].median())

#Drop rows where both dates are missing
df = df[~(df['OrderDate'].isna() & df['ShippedDate'].isna())]
print(len(df))

242


Handling Nulls, Fixing Fields and Standardization

The following code works perfectly fine but pandas gave me error that in future some things will be changed so the proper code to write that does the same effect is:
df.loc[:, 'OrderID'] = df['OrderID'].ffill() 
df.loc[:, 'CustomerID'] = df['CustomerID'].fillna("Unknown")
df.loc[:, 'ShipCity'] = df['ShipCity'].fillna("Unspecified")
df.loc[:, 'ShipCountry'] = df['ShipCountry'].str.strip().str.title() 
df.loc[:, 'ShipCity'] = df['ShipCity'].str.strip().str.title()
df.loc[:, 'ShippingCompany'] = df['ShippingCompany'].str.strip()

In [23]:
#To know which columns has null or missing values left
#print(df.isna().sum())
#Handle NULLS (varies from one case to another according to the client)
df['OrderID'] = df['OrderID'].fillna(method='ffill') # method = 'ffill' fills each null data with the previous data in the row before it
df['CustomerID'] = df['CustomerID'].fillna("Unknown")
df['ShipCity'] = df['ShipCity'].fillna("Unspecified")

#Standardize names
df['ShipCountry'] = df['ShipCountry'].str.strip().str.title() # .strip -> remove extra spaces from left or right, .title -> makes each first letter is capital
df['ShipCity'] = df['ShipCity'].str.strip().str.title()
df['ShippingCompany'] = df['ShippingCompany'].str.strip()

#Fix Specific cases
df.loc[df['ShippingCompany'].str.contains("Kiwilytics", na=False), 'ShippingCompany'] = "Kiwilytics Goods Shipping LLC."
print(df.isna().sum())

OrderID             0
CustomerID          0
OrderDate          16
ShippedDate        31
ShippingCost        0
ShipCountry         0
ShipCity            0
ShippingCompany     0
dtype: int64


/tmp/ipykernel_6368/1082303301.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['OrderID'] = df['OrderID'].fillna(method='ffill') # method = 'ffill' fills each null data with the previous data in the row before it
/tmp/ipykernel_6368/1082303301.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OrderID'] = df['OrderID'].fillna(method='ffill') # method = 'ffill' fills each null data with the previous data in the row before it
/tmp/ipykernel_6368/1082303301.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

Feature Engineering

In [26]:
# Days between order and shipment
df['DeliveryDays'] = (df['ShippedDate'] - df['OrderDate']).dt.days

# Flags
def get_status(x):
    if pd.isna(x):
        return "Unknown"
    elif x > 15:
        return "Late"
    else:
        return "On Time"
    
df['DeliveryStatus'] = df['DeliveryDays'].apply(get_status)

# Domestic vs International
def check_domestic(country):
    if country in domestic_countries:
        return "Yes"
    else:
        return "No"

domestic_countries = ['Germany']

df['IsDomestic'] = df['ShipCountry'].apply(check_domestic)

df.head()

,OrderID,CustomerID,OrderDate,ShippedDate,ShippingCost,ShipCountry,ShipCity,ShippingCompany,DeliveryDays,DeliveryStatus,IsDomestic
0,1000.0,C001,2025-05-17,2025-07-30,234.09,Germany,Hamburg,Kiwilytics Goods Shipping LLC.,74.0,Late,Yes
1,1001.0,C002,2025-01-26,2025-07-30,320.61,Canada,Montreal,UPS Worldwide,185.0,Late,No
2,1002.0,C003,2025-03-08,2025-07-30,165.17,Canada,Vancouver,FedEx Logistics,144.0,Late,No
3,1003.0,C004,2025-03-24,2025-07-30,12.55,Germany,Munich,Aramex International,128.0,Late,Yes
4,1004.0,C005,2025-04-15,2025-07-30,186.36,Canada,Vancouver,FedEx Logistics,106.0,Late,No


Grouping

In [27]:
avg_shipping_by_company = df.groupby('ShippingCompany')['ShippingCost'].mean()
print("Avg Shipping Cost by Company: \n")
print(avg_shipping_by_company, "\n")

Avg Shipping Cost by Company: 

ShippingCompany
Aramex International              255.897451
DHL Express                       228.160192
FedEx Logistics                   229.931321
Kiwilytics Goods Shipping LLC.    244.036829
UPS Worldwide                     259.419333
Name: ShippingCost, dtype: float64 



Export, Reporting and Wrap-up

In [28]:
#Export cleaned file
df.to_csv("cleaned_orders_final.csv",index = False)# index= False -> because i don't want indexing in the final clean sheet
#Final Summary
print("\n Final Dataset Snapshot::\n")
print(df.head())

print("\n Delivery Status Breakdown:\n")
print(df['DeliveryStatus'].value_counts())

print("\n Orders by Country:\n")
print(df['ShipCountry'].value_counts())

print("\n Orders by City:\n")
print(df['ShipCity'].value_counts())

print("\n Top 3 Shipping Companies:\n")
print(df['ShippingCompany'].value_counts().head(3))


 Final Dataset Snapshot::

   OrderID CustomerID  OrderDate ShippedDate  ShippingCost ShipCountry  \
0   1000.0       C001 2025-05-17  2025-07-30        234.09     Germany   
1   1001.0       C002 2025-01-26  2025-07-30        320.61      Canada   
2   1002.0       C003 2025-03-08  2025-07-30        165.17      Canada   
3   1003.0       C004 2025-03-24  2025-07-30         12.55     Germany   
4   1004.0       C005 2025-04-15  2025-07-30        186.36      Canada   

    ShipCity                 ShippingCompany  DeliveryDays DeliveryStatus  \
0    Hamburg  Kiwilytics Goods Shipping LLC.          74.0           Late   
1   Montreal                   UPS Worldwide         185.0           Late   
2  Vancouver                 FedEx Logistics         144.0           Late   
3     Munich            Aramex International         128.0           Late   
4  Vancouver                 FedEx Logistics         106.0           Late   

  IsDomestic  
0        Yes  
1         No  
2         No  
3   